# Online Feedback with LastMile Tracer

## Introduction
This notebook demonstrates how to use the LastMile Tracer library to log online feedback for generated content using OpenAI's GPT-3.5-turbo model. It guides you through the process of setting up the LastMile Tracer, creating and managing traces and spans, and associating user feedback with specific traces for better analysis and debugging.

In [1]:
!pip install lastmile-eval --upgrade
!pip install ipywidgets

In [4]:
# Setup Cell
# Import necessary modules from the lastmile_eval package
from lastmile_eval.rag.debugger.tracing import (
    get_lastmile_tracer,
)
from lastmile_eval.rag.debugger.api import (
    LastMileTracer)

# Replace the lastmile_api_token with your actual token
lastmile_api_token = "Enter Token here"

# Create a LastMileTracer instance with the specified project name and API token
tracer: LastMileTracer = get_lastmile_tracer(
    tracer_name="generate-riddle", # project name
    lastmile_api_token= lastmile_api_token,
)

2024-05-20 00:05:03,918 - Overriding of current TracerProvider is not allowed
2024-05-20 00:05:03,920 - Starting new HTTPS connection (1): lastmileai.dev:443


2024-05-20 00:05:04,023 - https://lastmileai.dev:443 "GET /api/evaluation_projects/list?name=generate-riddle HTTP/1.1" 200 350


Logging Online Feedback After Completing a Trace

In [10]:
from lastmile_eval.rag.debugger.api import LastMileTracer
from lastmile_eval.rag.debugger.tracing import get_lastmile_tracer
from lastmile_eval.rag.debugger.tracing.sdk import get_span_id, get_trace_id
import os
from openai import OpenAI


@tracer.start_as_current_span("generate_riddle")
def generate()-> str:
    """
    Generates a riddle using OpenAI's GPT-3.5-turbo model.
    If the OPENAI_API_KEY environment variable is not set, a default riddle is returned.
    The generated riddle is returned as a string.
    """
    riddle = ""
    if os.environ.get("OPENAI_API_KEY") is not None:
        response = OpenAI().chat.completions.create(messages = [{"role": "user", "content":"tell me a riddle"}], model = "gpt-4-turbo")
        riddle = response.choices[0].message.content or "debugging me can be quite a trick."
    else:
        riddle = "I'm a key that unlocks no doors, But fill me in, and power soars. In the realm where AI plays"

    # Store the span ID and trace ID associated with the generated riddle using the LastMileTracer's key-value store
    # This allows us to retrieve the span and trace IDs later when logging feedback for the specific riddle
    try:
        # Store the span id and the trace ID with the generated riddle
        tracer.store_trace_id(key=riddle, span_id=get_span_id(), trace_id= get_trace_id()) 
    except Exception as e:

        print(f'Exception Occured when storing TraceID: {e}')
    return riddle

In [ ]:
riddle = generate()

# We stored the spanID in the previous step, so we can retrieve it using the riddle as the key
trace_id, span_id = tracer.read_trace_id(key=riddle)

from IPython.display import display, clear_output
from ipywidgets import Button, HBox, Output

output = Output()

def get_and_store_user_feedback():
    def on_thumbsup_clicked(b):
        with output:
            clear_output()
            print("You selected: 👍")
            tracer.log_feedback(span_id=span_id, trace_id=trace_id, feedback="👍")  

    def on_thumbsdown_clicked(b):
        with output:
            clear_output()
            print("You selected: 👎")
            tracer.log_feedback(span_id=span_id, trace_id=trace_id, feedback="👎")  

    thumbsup_button = Button(description="👍", button_style="success")
    thumbsup_button.on_click(on_thumbsup_clicked)

    thumbsdown_button = Button(description="👎", button_style="danger")
    thumbsdown_button.on_click(on_thumbsdown_clicked)

    display(HBox([thumbsup_button, thumbsdown_button]))
    display(output)
print(f'\n Generated Riddle:\n{riddle}\n Give it a thumbs up or thumbs down to provide feedback.\n')
get_and_store_user_feedback()